<a href="https://colab.research.google.com/github/zvryab/FYP/blob/main/FYP_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community

!pip install --upgrade pip setuptools wheel

Terminal Text:

/curl -fsSL https://ollama.com/install.sh | sh /ollama serve & ollama run llama2 /ollama pull llama3.1

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama3.1",  # or whicever model I pulled, use 3.3 for better results
    base_url="https://d06d-134-151-21-61.ngrok-free.app",
    temperature=0,
)


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [ ]:
# Initialize memory to store conversation context
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
!pip install psycopg2 sqlalchemy langchain ollama

In [ ]:
!pip install langchain langchain-community sqlalchemy psycopg2-binary

from sqlalchemy import create_engine
from langchain_community.llms import Ollama
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase
from langchain.agents.agent_types import AgentType

# Step 1: Set up database engine (already working)
DB_HOST = "7.tcp.eu.ngrok.io"
DB_PORT = "11662"
DB_NAME = "sampledb"
DB_USER = "postgres"

engine = create_engine(f"postgresql://{DB_USER}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Step 2: Set up LangChain SQLDatabase object
db = SQLDatabase(engine)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
from langchain.chains import LLMChain

In [ ]:
# Step 3: Set up Ollama LLM
llm = Ollama(model="llama3.1")  # or "mistral", etc.

prompt = PromptTemplate(
    input_variables=["input", "table_info"],
    template=_mysql_prompt + PROMPT_SUFFIX,
)

llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

from langchain.prompts import PromptTemplate

custom_sql_prompt = PromptTemplate(
    input_variables=["input", "table_info"],
    template="""
You are an expert SQL generator designed to interact with a PostgreSQL database.
Generate a syntactically correct SQL query to answer the given natural language question.

Only output the SQL query. Do not include explanations, comments, or intermediate steps.

Schema:
{table_info}

Question:
{input}

SQL Query:
"""
)

# Step 4: Create the SQL agent
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    memory=memory
    #prefix= custom_prefix
)


In [ ]:
!pip install gradio

In [ ]:
!pip install matplotlib plotly

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import text

def visualize_query(query):
    try:
        # Execute the query and load results into a DataFrame
        with engine.connect() as conn:
            result = conn.execute(text(query))
            data = result.fetchall()
            columns = result.keys()

        # Create a DataFrame from the query result
        df = pd.DataFrame(data, columns=columns)

        # Basic visualization: Bar chart if there are two columns (label and value)
        if df.shape[1] == 2:
            plt.figure(figsize=(8, 5))
            plt.bar(df.iloc[:, 0], df.iloc[:, 1], color='skyblue')
            plt.title("Query Result Visualization")
            plt.xlabel(df.columns[0])
            plt.ylabel(df.columns[1])
            plt.xticks(rotation=45)
            plt.tight_layout()

            # Save the plot as an image
            plt.savefig("query_result.png")
            plt.close()
            return "query_result.png"
        else:
            return "Visualization not supported for this query format."
    except Exception as e:
        return f"Error in visualization: {str(e)}"


In [ ]:
import gradio as gr

def sql_query_agent(input_text, output_type):
    try:
        # Replace with the function you already have to get SQL results
        result = agent_executor.run(input_text)

                # If the output type is "Visualization", generate a plot
        if output_type == "Graph":
            image_path = visualize_query(result)
            if image_path.startswith("Error"):
                return image_path
            return gr.Image.update(value=image_path)

        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

# Create the Gradio interface
iface = gr.Interface(
    fn=sql_query_agent,
    inputs=[
        gr.Textbox(label="SQL Query"),
        gr.Dropdown(choices=["Text", "Graph"], label="Output Type", value="Text")
    ],
    outputs=gr.Textbox(label="Query Result"),
    title="FYP: SQL Query Agent with Visualization",
    description="Ask a question about your database, and I will generate and execute the SQL query for you. The agent remembers the context of previous questions.",
)


In [ ]:
iface.launch()